🧩 Célula 1 – Importação de biblioteca

In [97]:
import os
import pandas as pd
from shutil import move
from datetime import datetime, timedelta

✅ Fazer o backup com a data atual


In [98]:
# 📁 Caminhos e nomes de arquivos
pasta_origem = "C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Vendas"
nome_arquivo = "Arquivo_Consolidado.xlsx"

# 📅 Data atual formatada
data_hoje = datetime.now().strftime("%Y-%m-%d")

# 📁 Pasta de backup
pasta_backup = os.path.join(pasta_origem, "Backup")
os.makedirs(pasta_backup, exist_ok=True)

# 🧭 Verifica e move o arquivo consolidado anterior
if os.path.exists(caminho_consolidado):
    nome_backup = f"Backup_{data_hoje}_{nome_arquivo}"
    caminho_backup = os.path.join(pasta_backup, nome_backup)
    move(caminho_consolidado, caminho_backup)
    print(f"📦 Arquivo consolidado anterior movido para backup: {caminho_backup}")

# 🕒 Limpeza de backups antigos (mantém últimos 7 dias)
limite_dias = 7
data_limite = datetime.now() - timedelta(days=limite_dias)

for arquivo in os.listdir(pasta_backup):
    caminho_arquivo = os.path.join(pasta_backup, arquivo)

    if arquivo.startswith("Backup_") and arquivo.endswith(".xlsx"):
        try:
            data_str = arquivo.split("_")[1]
            data_arquivo = datetime.strptime(data_str, "%Y-%m-%d")

            if data_arquivo < data_limite:
                os.remove(caminho_arquivo)
                print(f"🧹 Backup antigo removido: {arquivo}")
        except Exception as e:
            print(f"⚠️ Não foi possível processar o arquivo '{arquivo}': {e}")

📦 Arquivo consolidado anterior movido para backup: C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Vendas\Backup\Backup_2025-08-11_Arquivo_Consolidado.xlsx
🧹 Backup antigo removido: Backup_2025-08-04_Arquivo_Consolidado.xlsx


📂 1. Definição do diretório e leitura dos arquivo

In [99]:

caminho_consolidado = os.path.join(pasta_origem, nome_arquivo)
caminho_log = os.path.join(pasta_origem, "Log_Duplicatas.txt")

# 🗂️ Apagar o arquivo de log
if os.path.exists(caminho_log):
    os.remove(caminho_log)
    print("🗑️ Log anterior removido.")
    
# 📦 Lista para armazenar os DataFrames
dados = []

# 🧩 Função para verificar arquivos
def verificar_arquivos(pasta_origem, nome_arquivo):
    arquivos_encontrados = [
        arquivo for arquivo in os.listdir(pasta_origem)
        if arquivo.endswith('.xlsx') and arquivo != nome_arquivo
    ]

    if not arquivos_encontrados:
        print("⚠️ Nenhum arquivo de vendas encontrado para consolidar.")
        print("✅ O arquivo consolidado existente foi mantido.")
        return None

    print("📂 Arquivos encontrados para consolidação:")
    for arquivo in arquivos_encontrados:
        print("   -", arquivo)

    return arquivos_encontrados

# 🧩 Função para validar layout
def validar_layout(arquivos_encontrados, layout_esperado, pasta_origem):
    arquivos_invalidos = []

    for arquivo in arquivos_encontrados:
        caminho_arquivo = os.path.join(pasta_origem, arquivo)
        df = pd.read_excel(caminho_arquivo)
        colunas_encontradas = list(df.columns)

        if sorted(colunas_encontradas) != sorted(layout_esperado):
            arquivos_invalidos.append((arquivo, colunas_encontradas))

    if arquivos_invalidos:
        for arquivo, colunas in arquivos_invalidos:
            print(f"\n❌ ERRO: O arquivo ->'{arquivo}'<- está fora do layout esperado.")
            print("📌 Colunas encontradas:")
            print("   ", colunas)
            print("✅ Colunas esperadas:")
            print("   ", layout_esperado)
        print("\n🚫 Consolidação interrompida. Corrija os arquivos e execute novamente.")
        return False

    return True

# 📋 Define o layout esperado
layout_esperado = [
    'Data', 'Id Venda', 'IdCategoria', 'Categoria', 'IdPais', 'Pais',
    'idProduto', 'Produto', 'Custo fabricacao', 'Unidades Vendidas',
    'Preco venda', 'Vendas total', 'Descontos'
]

# 🔍 Verifica arquivos
arquivos_encontrados = verificar_arquivos(pasta_origem, nome_arquivo)

if arquivos_encontrados is None:
    print("⏹️ Execução encerrada.")
else:
    if not validar_layout(arquivos_encontrados, layout_esperado, pasta_origem):
        print("⏹️ Execução encerrada.")
    else:
        print("✅ Todos os arquivos estão válidos. Pronto para consolidar!")

        # ✅ Leitura e concatenação
        for arquivo in arquivos_encontrados:
            caminho_arquivo = os.path.join(pasta_origem, arquivo)
            df = pd.read_excel(caminho_arquivo)
            df['Arquivo_Origem'] = arquivo
            dados.append(df)

        # 🔗 Concatena todos os DataFrames válidos
        Dados_consolidado = pd.concat(dados, ignore_index=True)

        # 📊 Contagem antes da remoção de duplicatas
        total_antes = Dados_consolidado.shape[0]

        # 🔁 Identifica duplicatas (sem considerar a coluna 'Arquivo_Origem')
        colunas_para_verificar = [col for col in Dados_consolidado.columns if col != 'Arquivo_Origem']
        duplicatas = Dados_consolidado[Dados_consolidado.duplicated(subset=colunas_para_verificar, keep=False)]

        # 📝 Salva duplicatas no log
        if not duplicatas.empty:
            with open(caminho_log, "w", encoding="utf-8") as log:
                log.write("🔁 REGISTROS DUPLICADOS ENCONTRADOS:\n\n")
                log.write(duplicatas.to_string(index=False))
            print(f"📝 Log de duplicatas salvo em: {caminho_log}")
        else:
            print("✅ Nenhuma duplicata encontrada. Nenhum log gerado.")

        # 🧹 Remove duplicidades mantendo o primeiro registro
        Dados_consolidado.drop_duplicates(subset=colunas_para_verificar, inplace=True)

        # 📉 Contagem depois da remoção
        total_depois = Dados_consolidado.shape[0]
        duplicatas_removidas = total_antes - total_depois

        # 🗑️ Remove o arquivo consolidado anterior, se existir
        if os.path.exists(caminho_consolidado):
            os.remove(caminho_consolidado)
            print("🗑️ Arquivo consolidado anterior removido.")

        # 💾 Salva o novo consolidado
        Dados_consolidado.to_excel(caminho_consolidado, index=False)

        # 📣 Mensagens finais
        print("✅ Novo arquivo consolidado criado em:", caminho_consolidado)
        print(f"📊 Registros antes da remoção de duplicatas: {total_antes}")
        print(f"📉 Registros após remoção de duplicatas: {total_depois}")
        print(f"🧹 Total de duplicatas removidas: {duplicatas_removidas}")

        # 📄 Exibe o conteúdo do log de duplicatas no console
        if os.path.exists(caminho_log):
            print("\n📄 Conteúdo do Log_Duplicatas.txt:")
            with open(caminho_log, "r", encoding="utf-8") as log:
                print(log.read())
# Verificar estatisticas do DataFrame consolidado
        print("\n📊 Estatísticas do DataFrame consolidado:")
# Data minima e máxima
        print("📅 Data mínima:", Dados_consolidado['Data'].min())
        print("📅 Data máxima:", Dados_consolidado['Data'].max())
# Agrupar e conta pela data
        Total_Registros = Dados_consolidado.groupby('Data').size().reset_index(name='Quantidade')
        print("\n📅 Visão por Data e Qtd:", Total_Registros)


🗑️ Log anterior removido.
📂 Arquivos encontrados para consolidação:
   - Vendas2021.xlsx
   - Vendas2026.xlsx
✅ Todos os arquivos estão válidos. Pronto para consolidar!
📝 Log de duplicatas salvo em: C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Vendas\Log_Duplicatas.txt
✅ Novo arquivo consolidado criado em: C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Vendas\Arquivo_Consolidado.xlsx
📊 Registros antes da remoção de duplicatas: 33
📉 Registros após remoção de duplicatas: 31
🧹 Total de duplicatas removidas: 2

📄 Conteúdo do Log_Duplicatas.txt:
🔁 REGISTROS DUPLICADOS ENCONTRADOS:

      Data  Id Venda  IdCategoria  Categoria  IdPais   Pais  idProduto    Produto  Custo fabricacao  Unidades Vendidas  Preco venda  Vendas total  Descontos  Arquivo_Origem
2021-04-12       760            4 Acessórios     638 México          2 Braceletes               250                939           15         14085      908.4 Vendas2021.xlsx
2021-04-12    